In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

import utils
import config


def main():

    # if read reserved model 
    if not config.load_model:
        print('loading model...')
        # load training data
        train_images, train_labels = utils.load_data(config.train_data_dir)

        # display image randomly
        utils.display_traffic_signs(train_images, train_labels)

        # process training model for input
        X_train, y_train = utils.process_data(train_images, train_labels)

        print('training model...')
        # build CNN
        cnn_model = utils.build_cnn()

        # if image augmentation
        if not config.data_augmentation:
            print('no image augmentation...')

            # traning model
            cnn_model.fit(X_train, y_train,
                          batch_size=config.batch_size,
                          epochs=config.epochs)
        else:
            print('image augmentation...')
            
            datagen = ImageDataGenerator(
                width_shift_range=0.1,  
                height_shift_range=0.1,  
                horizontal_flip=True  
            )

            # augmentation on training data
            datagen.fit(X_train)
            cnn_model.fit_generator(datagen.flow(X_train, y_train, batch_size=config.batch_size),
                                    epochs=config.epochs,
                                    workers=4)
        # reserve trained model 
        cnn_model.save(config.model_file)

    else:
        print('loading trained model...')
        if os.path.exists(config.model_file):
            cnn_model = load_model(config.model_file)
        else:
            print('{}model file unexist'.format(config.model_file))
            return

    print('loading test data...')
    test_images, test_labels = utils.load_data(config.test_data_dir)

    # process traning data for model input
    X_test, y_test = utils.process_data(test_images, test_labels)
    test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
    print('test loss: {:.6f}，test accuracy：{:.2f}'.format(test_loss, test_acc))


if __name__ == '__main__':
    main()


Using TensorFlow backend.


加载训练数据...
共有4575个图像被加载。


<Figure size 1500x800 with 1 Axes>

<Figure size 1000x1000 with 62 Axes>

训练模型...
构建CNN
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activat

<Figure size 1500x800 with 1 Axes>

2520/2520 [==============================] - 1s 435us/step
测试loss: 0.197463，测试准确率：0.96
